# This Notebook will be used for the Capstone Project - The Battle of Neighborhoods (Week 2)

## Contents
1. [Introduction/ Business Problem](#intro)
2. [Data Description](#data_desc)
3. [Methodology](#methodology)
4. [Analysis](#analysis)
5. [Results](#results)
6. [Discussion](#discussion)
7. [Conclusion](#conclusion)

## 1. Introduction/ Business Problem <a name="intro"></a>

When it comes to the Pacific North West, Seattle is the party town. The go to place for night life. When it commes to adjoining
cities like Bellevue, the only lights are the street lights. There is virtually no night life, even though all indicators like concentration of Bars, Median Household Income, party loving crowd is comparable to Seattle. This Project aims to finding the right neighborhoods to setup Night Clubs, so people from Bellevue and adjoining areas dont missout on the fun.
People who would be interested in this idea would not just be the Bar and Nightclub industry. But also Restaurants, Housing and Tourism will be greatle impacted. It is to note here that the the law and order problems might go up, which will increase the demand for law enforcement officers required.<br>
**Bar and Nighclub Industry**: The industry will see a jummp in footfall in areas where the Night Club comes up. We could say existing Bars and breweries etc. might get impacted in short term. But in long term it will lead to an increased footfall from adjoining suburbs hence improved business.<br>
**Restaurants**: Increased footfall will help the restaurants especially the ones running late into night bring in the after party crowd.<br>
**Housing**: An improved nightlife with multiple obtions will lead party going crowd to stay around such neighborhoods or close to these, for increased access. Opening up the housing sector to more customers driving the rents and housing prices higher.<br>
**Tourism**: Tourism in Bellevue runs high in the short Summer compared to the long winters. An improved night life will drive up the tourism specially folks visiting the nearby Skiing spots in winter.<br>
**Law Enforcement**: The demand for law enforcement will go up with the increase in footfall leading to higher recruitment of officers, which can be a positive from a job demand perspective.<br>
**Cabs and Parking Facilities**: THe demand will go specifically because the number of people taking Cabs will go up and also the number of nightime parkings will shoot. Bringing in additional revenue for the industry.<br>

## 2. Data Desciption <a name="data_desc"></a>

When it comes to data, I will check areas in Seattle with high nightclubs to find what other late running shops might be influencing the presence of the night clubs or vice versa. Like presence of Bars and Restaurants etc. Simialr concentration I will try to find in Bellevue and see which areas specifically come close to such markers. This would help in deciding the right neighborhoods for setting up shops. <br>
I will be playing with the Four Square data in the following way: <br>
1. I would be looking up Night Clubs in Seattle and then look for Bars and Restaurants in close proximity. These bars and restaurants would be filtered for late hours.
2. I would define the cluster based on night life excluding Night clubs for a fair comparison with Bellevue Neighborhoods <br>
3. Next i would try to get the night life in Bellevue neightborhoods, I might have to adjust the running hours slightly to take into account the non existent night life. Assuming places closing not so late, would close later with increased late night business.<br>
4. Based on the above findings I will find neighborhoods which are suitable for setting up Night Clubs.
5. If multiple such neighborhoods are found then I will look for secondary requirements like proximity to Hotels, Parking Facilities and Law Enforcment agencies.<br>
6. I also wanted to correlate with Housing so we are slightly away from high concentration of Housing and not very far. I am yet to find to find any such dataset to use. Would love to include it if if i find something.

### Importing the required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
# from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Defining Four Square Credentials

In [2]:
CLIENT_ID = 'P3Q0EEQIF5A3033ZLP1RLQDIWIUCANZYF3KQZ4LAZC4AAZFA' # my Foursquare ID
CLIENT_SECRET = '32RTBTOZRF1DMCPHC1BWVO23B2Q4D4DCX2U2TWHWIQATG1JZ' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Function to get Nightclub data for Seattle

In [3]:
def getocationData(search_query,LIMIT,sw,ne):
    # url = 'https://api.foursquare.com/v2/search/recommendations?client_id={}&client_secret={}&near={}&v={}&query={}&limit={}'.format(
    #     CLIENT_ID, CLIENT_SECRET, city, VERSION, search_query, LIMIT)
    url='https://api.foursquare.com/v2/search/recommendations?client_id={}&client_secret={}&v={}&query={}&limit={}&sw={}&ne={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, search_query, LIMIT, sw, ne)
    results=requests.get(url).json()
    return results

### Function to convert the JSON to Data Frame

In [4]:
def convert2Df(results):
    # assign relevant part of JSON to venues
    df0 = pd.DataFrame(results['response']['group']['results'])

    # tranform venues into a dataframe
    df1 = pd.json_normalize(df0['venue'])
    return df1

### Get night clubs in Seattle

In [5]:
search_query='Nightclub'
LIMIT=100
sw='47.56471241397957,-122.46485710144042'
ne='47.66099410795319,-122.2550868988037'

result=getocationData(search_query,LIMIT,sw,ne)
nc_df=convert2Df(result)
nc_df.head()

,id,name,categories,dislike,ok,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,40cf8d80f964a52026011fe3,Neumos,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",False,False,925 E Pike St,at 10th Ave,47.613930,-122.319625,"[{'label': 'display', 'lat': 47.61393031118657...",98122,US,Seattle,WA,United States,"[925 E Pike St (at 10th Ave), Seattle, WA 9812...",47114665,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5300602411d271adabb2fde2,Kremwerk,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",False,False,1809 Minor Ave Ste 10,NaN,47.616926,-122.330880,"[{'label': 'display', 'lat': 47.61692560525665...",98101,US,Seattle,WA,United States,"[1809 Minor Ave Ste 10, Seattle, WA 98101, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5708aba9498e4c3d466d6602,The Underground,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",False,False,122 S Washington St,NaN,47.600898,-122.333032,"[{'label': 'display', 'lat': 47.60089818452381...",98104,US,Seattle,WA,United States,"[122 S Washington St, Seattle, WA 98104, Unite...",NaN,Pioneer Square,NaN,NaN,NaN,NaN,NaN,NaN
3,4c660a52d6f676b083fdd26c,Club Sur,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",False,False,2901 1st Ave S,S Forest St.,47.577530,-122.334603,"[{'label': 'display', 'lat': 47.57752950659195...",98134,US,Seattle,WA,United States,"[2901 1st Ave S (S Forest St.), Seattle, WA 98...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,412bd680f964a520b80c1fe3,Mercury @ MachineWerks,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",False,False,1009 E Union St,10th Ave E,47.612659,-122.318865,"[{'label': 'display', 'lat': 47.61265941325798...",98122,US,Seattle,WA,United States,"[1009 E Union St (10th Ave E), Seattle, WA 981...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Functions to clean up the DF

In [6]:
# extract venue category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def cleanupDf(df):
    # get required columns
    filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
    filt_df = df.loc[:, filtered_columns]

    # filter the category for each row
    filt_df['categories'] = filt_df.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    filt_df.columns = [column.split('.')[-1] for column in filt_df.columns]
    return filt_df

In [7]:
#cleaning the df
nc_filt_df=cleanupDf(nc_df)
nc_filt_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Neumos,Nightclub,925 E Pike St,at 10th Ave,47.613930,-122.319625,"[{'label': 'display', 'lat': 47.61393031118657...",98122,US,Seattle,WA,United States,"[925 E Pike St (at 10th Ave), Seattle, WA 9812...",NaN,40cf8d80f964a52026011fe3
1,Kremwerk,Nightclub,1809 Minor Ave Ste 10,NaN,47.616926,-122.330880,"[{'label': 'display', 'lat': 47.61692560525665...",98101,US,Seattle,WA,United States,"[1809 Minor Ave Ste 10, Seattle, WA 98101, Uni...",NaN,5300602411d271adabb2fde2
2,The Underground,Nightclub,122 S Washington St,NaN,47.600898,-122.333032,"[{'label': 'display', 'lat': 47.60089818452381...",98104,US,Seattle,WA,United States,"[122 S Washington St, Seattle, WA 98104, Unite...",Pioneer Square,5708aba9498e4c3d466d6602
3,Club Sur,Nightclub,2901 1st Ave S,S Forest St.,47.577530,-122.334603,"[{'label': 'display', 'lat': 47.57752950659195...",98134,US,Seattle,WA,United States,"[2901 1st Ave S (S Forest St.), Seattle, WA 98...",NaN,4c660a52d6f676b083fdd26c
4,Mercury @ MachineWerks,Nightclub,1009 E Union St,10th Ave E,47.612659,-122.318865,"[{'label': 'display', 'lat': 47.61265941325798...",98122,US,Seattle,WA,United States,"[1009 E Union St (10th Ave E), Seattle, WA 981...",NaN,412bd680f964a520b80c1fe3


### Function to Neighbourhood and Suburb

In [8]:
def getNeigh(row):
    lat_lon=str(row['lat'])+", "+str(row['lng'])
    location = geolocator.reverse(lat_lon)
    try:
        row['commercial']=location.raw['address']['commercial']
    except:
        pass
    try:
        row['neighbourhood']=location.raw['address']['neighbourhood']
    except:
        pass
    try:
        row['suburb']=location.raw['address']['suburb']
    except:
        pass
    return row

### Getting Night Club neighbourhoods

In [9]:
geolocator = Nominatim(user_agent="foursquare_agent")
nc_neigh_df=nc_filt_df.apply(getNeigh, axis=1)
nc_neigh_df.drop(['labeledLatLngs','postalCode','cc','city','state','country','formattedAddress','address','id','neighborhood'], axis=1,\
                    inplace=True)

In [10]:
nc_neigh_df.groupby(['suburb']).count()

,categories,commercial,crossStreet,lat,lng,name,neighbourhood
suburb,,,,,,,
Belltown,5,1,3,5,5,5,4
Capitol Hill,3,0,1,3,3,3,2
First Hill,15,0,10,15,15,15,14
Fremont,2,0,2,2,2,2,0
International District/Chinatown,13,0,6,13,13,13,9
Mount Baker,1,0,0,1,1,1,0


In [11]:
nc_neigh_df.groupby(['neighbourhood']).count()

,categories,commercial,crossStreet,lat,lng,name,suburb
neighbourhood,,,,,,,
Central Business District,14,0,11,14,14,14,14
Montlake,1,0,0,1,1,1,1
South Lake Union,5,0,3,5,5,5,3
West Edge,11,0,6,11,11,11,11


In [12]:
nc_neigh_df.groupby(['commercial']).count()

,categories,crossStreet,lat,lng,name,neighbourhood,suburb
commercial,,,,,,,
Pike Place Market Area,1,0,1,1,1,0,1


### Looking at above, **suburbs** seems more reliable than Neighbourhoods or Commercial hence we will use the former

### Dropping Nan rows for Suburb column

In [13]:
nc_neigh_df=nc_neigh_df.dropna(subset=['suburb']).reset_index(drop=True)
nc_neigh_df.head()

,categories,commercial,crossStreet,lat,lng,name,neighbourhood,suburb
0,Nightclub,NaN,at 10th Ave,47.613930,-122.319625,Neumos,Central Business District,First Hill
1,Nightclub,NaN,NaN,47.616926,-122.330880,Kremwerk,South Lake Union,First Hill
2,Nightclub,NaN,NaN,47.600898,-122.333032,The Underground,West Edge,International District/Chinatown
3,Nightclub,NaN,S Forest St.,47.577530,-122.334603,Club Sur,NaN,International District/Chinatown
4,Nightclub,NaN,10th Ave E,47.612659,-122.318865,Mercury @ MachineWerks,Central Business District,First Hill


### Function to get Neigborhoods Bounds

In [14]:
def getBounds(loc):
    location = geolocator.geocode(loc)
    sw=location.raw['boundingbox'][0]+","+location.raw['boundingbox'][2]
    ne=location.raw['boundingbox'][1]+","+location.raw['boundingbox'][3]
    return sw+';'+ne

In [22]:
# Getting Nightlife Venues per Neighborhood
nl_df2=pd.DataFrame()
search_query='Nightlife'
city=', Seattle, WA'
LIMIT=1
for neigh in nc_neigh_df['suburb'].unique():
    print(neigh)
    sw,ne=getBounds(neigh+city).split(';')
    print(sw,ne)
    result=getocationData(search_query,LIMIT,sw,ne)
    try:
        nl_df=convert2Df(result)
    except:
        print("Location with issue:"+neigh)
    nl_df['location.suburb']=neigh
    nl_df2=pd.concat([nl_df,nl_df2],ignore_index=True)
nl_df2.shape

First Hill
47.5893053,-122.3449839 47.6293053,-122.3049839
International District/Chinatown
47.5977888,-122.3281795 47.5989367,-122.3278264
Location with issue:International District/Chinatown
Capitol Hill
47.6038307,-122.3383689 47.6438307,-122.2983689
Belltown
47.5932311,-122.3653607 47.6332311,-122.3253607
Mount Baker
47.5585522,-122.3095262 47.5985522,-122.2695262
Fremont
47.6304529,-122.3699861 47.6704529,-122.3299861


(6, 23)

### The above shows we cant get Night Life venues for each Neighborhood by lat lon pairs since we are missing Chinatown for some reason, even though it has a majority of night clubs

In [30]:
search_query='Nightlife'
LIMIT=500
sw='47.56471241397957,-122.46485710144042'
ne='47.66099410795319,-122.2550868988037'

result=getocationData(search_query,LIMIT,sw,ne)
nl_df=convert2Df(result)
nl_df.head()

,id,name,categories,dislike,ok,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,54027198498ed729c02ed6c6,Schilling Cider House,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,False,708 34th Ave NE,at Fremont Ave N,47.649558,-122.349232,"[{'label': 'display', 'lat': 47.64955772631211...",98109,US,Seattle,WA,United States,"[708 34th Ave NE (at Fremont Ave N), Seattle, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40b13b00f964a52081f61ee3,Fremont Brewing Company,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,False,1050 N 34th St,at Woodland Park Ave N,47.648974,-122.344491,"[{'label': 'display', 'lat': 47.64897392628347...",98103,US,Seattle,WA,United States,"[1050 N 34th St (at Woodland Park Ave N), Seat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53cb3d97498eaf7d403ef4f3,Holy Mountain Brewing Company,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,False,1421 Elliott Ave W,at W Galer St,47.630795,-122.374597,"[{'label': 'display', 'lat': 47.63079465756503...",98119,US,Seattle,WA,United States,"[1421 Elliott Ave W (at W Galer St), Seattle, ...",92249185,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,54f8d7cd498e011cb93ae3c9,Cloudburst Brewing,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,False,2116 Western Ave,NaN,47.611565,-122.345212,"[{'label': 'display', 'lat': 47.61156493673511...",98121,US,Seattle,WA,United States,"[2116 Western Ave, Seattle, WA 98121, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55b2e221498e74c43d4ecdfc,Rooftop Brewing Company,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,False,1220 W Nickerson St,13th Ave NW,47.655766,-122.373127,"[{'label': 'display', 'lat': 47.65576624672368...",98119,US,Seattle,WA,United States,"[1220 W Nickerson St (13th Ave NW), Seattle, W...",NaN,North Queen Anne,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# cleaning up the df
nl_filt_df=cleanupDf(nl_df)
nl_filt_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Schilling Cider House,Bar,708 34th Ave NE,at Fremont Ave N,47.649558,-122.349232,"[{'label': 'display', 'lat': 47.64955772631211...",98109,US,Seattle,WA,United States,"[708 34th Ave NE (at Fremont Ave N), Seattle, ...",NaN,54027198498ed729c02ed6c6
1,Fremont Brewing Company,Brewery,1050 N 34th St,at Woodland Park Ave N,47.648974,-122.344491,"[{'label': 'display', 'lat': 47.64897392628347...",98103,US,Seattle,WA,United States,"[1050 N 34th St (at Woodland Park Ave N), Seat...",NaN,40b13b00f964a52081f61ee3
2,Holy Mountain Brewing Company,Brewery,1421 Elliott Ave W,at W Galer St,47.630795,-122.374597,"[{'label': 'display', 'lat': 47.63079465756503...",98119,US,Seattle,WA,United States,"[1421 Elliott Ave W (at W Galer St), Seattle, ...",NaN,53cb3d97498eaf7d403ef4f3
3,Cloudburst Brewing,Brewery,2116 Western Ave,NaN,47.611565,-122.345212,"[{'label': 'display', 'lat': 47.61156493673511...",98121,US,Seattle,WA,United States,"[2116 Western Ave, Seattle, WA 98121, United S...",NaN,54f8d7cd498e011cb93ae3c9
4,Rooftop Brewing Company,Brewery,1220 W Nickerson St,13th Ave NW,47.655766,-122.373127,"[{'label': 'display', 'lat': 47.65576624672368...",98119,US,Seattle,WA,United States,"[1220 W Nickerson St (13th Ave NW), Seattle, W...",North Queen Anne,55b2e221498e74c43d4ecdfc


### Getting Night Club neighbourhoods

In [32]:
nl_neigh_df=nl_filt_df.apply(getNeigh, axis=1)
nl_neigh_df.drop(['labeledLatLngs','postalCode','cc','city','state','country','formattedAddress','address','id','neighborhood'], axis=1,\
                    inplace=True)

In [33]:
nl_neigh_df.shape

(100, 8)

### drop all but those suburbs where Night Clubs are present and Night Clubs from Night Life list

In [34]:
nl_neigh_df2=nl_neigh_df[nl_neigh_df['suburb'].isin(list(nc_neigh_df['suburb'].unique()))]
nl_neigh_df2=nl_neigh_df2[~nl_neigh_df['name'].isin(list(nc_neigh_df['name'].unique()))]
nl_neigh_df2.shape

C:\Users\eutsaga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(76, 8)

In [103]:
# merging few categories 
nl_neigh_df2.replace(['Beer Bar','Gay Bar','Pub'],['Bar','Bar','Bar'],inplace=True)
nl_neigh_df2=nl_neigh_df2=nl_neigh_df2[~nl_neigh_df['categories'].isin(['Lounge','Other Nightlife'])]
nl_neigh_df2.groupby('categories').count()

C:\Users\eutsaga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,commercial,crossStreet,lat,lng,name,neighbourhood,suburb
categories,,,,,,,
Bar,1,16,25,25,25,17,25
Brewery,1,4,6,6,6,2,6
Cocktail Bar,3,10,18,18,18,12,18
Dive Bar,0,1,1,1,1,1,1
Gastropub,0,3,3,3,3,2,3
Hotel Bar,2,1,2,2,2,0,2
Karaoke Bar,0,1,1,1,1,1,1
Speakeasy,0,5,5,5,5,5,5
Sports Bar,0,1,1,1,1,1,1


In [104]:
nl_neigh_df2.groupby('suburb').count()

,categories,commercial,crossStreet,lat,lng,name,neighbourhood
suburb,,,,,,,
Belltown,20,8,11,20,20,20,12
Capitol Hill,15,0,10,15,15,15,10
First Hill,20,0,16,20,20,20,17
Fremont,8,0,5,8,8,8,1
International District/Chinatown,6,0,5,6,6,6,5


### Plotting the Night clubs vs other Night life places in seattle area

In [105]:
address = 'Seattle, WA'

# geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.6038321 -122.3300624


In [106]:
night_life_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Seattle

# # add a red circle marker to represent the Conrad Hotel
# folium.features.CircleMarker(
#     [latitude, longitude],
#     radius=10,
#     color='red',
#     popup='Conrad Hotel',
#     fill = True,
#     fill_color = 'red',
#     fill_opacity = 0.6
# ).add_to(venues_map)
# add the Venues as blue circle
for lat, lng, label in zip(nc_neigh_df.lat, nc_neigh_df.lng, nc_neigh_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(night_life_map)
    
    
# add the Venues as blue circle
for lat, lng, label in zip(nl_neigh_df2.lat, nl_neigh_df2.lng, nl_neigh_df2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(night_life_map)

# display map
night_life_map

In [107]:
nl_neigh_df2.groupby('categories').describe()

lat                                                        \
             count       mean       std        min        25%        50%   
categories                                                                 
Bar           25.0  47.622315  0.013084  47.613081  47.614074  47.616650   
Brewery        6.0  47.618781  0.028351  47.576040  47.610084  47.612893   
Cocktail Bar  18.0  47.615842  0.012151  47.600116  47.609161  47.614685   
Dive Bar       1.0  47.615173       NaN  47.615173  47.615173  47.615173   
Gastropub      3.0  47.614678  0.000669  47.614042  47.614329  47.614615   
Hotel Bar      2.0  47.612634  0.000013  47.612625  47.612630  47.612634   
Karaoke Bar    1.0  47.615287       NaN  47.615287  47.615287  47.615287   
Speakeasy      5.0  47.613189  0.002639  47.609512  47.613141  47.613150   
Sports Bar     1.0  47.591351       NaN  47.591351  47.591351  47.591351   
Wine Bar       7.0  47.617597  0.017475  47.599290  47.609251  47.614940   

                                     lng                                    \
                    75%        max count        mean       std         min   
categories                                                                   
Bar           47.622276  47.651681  25.0 -122.330802  0.014571 -122.354257   
Brewery       47.640286  47.652296   6.0 -122.341082  0.011043 -122.355560   
Cocktail Bar  47.623255  47.652662  18.0 -122.329728  0.012644 -122.356647   
Dive Bar      47.615173  47.615173   1.0 -122.323052       NaN -122.323052   
Gastropub     47.614996  47.615376   3.0 -122.327709  0.019183 -122.349658   
Hotel Bar     47.612638  47.612643   2.0 -122.352806  0.000018 -122.352819   
Karaoke Bar   47.615287  47.615287   1.0 -122.320031       NaN -122.320031   
Speakeasy     47.613169  47.616975   5.0 -122.326292  0.011584 -122.345083   
Sports Bar    47.591351  47.591351   1.0 -122.332422       NaN -122.332422   
Wine Bar      47.618183  47.654080   7.0 -122.332453  0.011092 -122.344408   

                                                              
                     25%         50%         75%         max  
categories                                                    
Bar          -122.343842 -122.327645 -122.317334 -122.307220  
Brewery      -122.345032 -122.343766 -122.337330 -122.322761  
Cocktail Bar -122.339784 -122.328516 -122.320928 -122.312512  
Dive Bar     -122.323052 -122.323052 -122.323052 -122.323052  
Gastropub    -122.334486 -122.319314 -122.316734 -122.314155  
Hotel Bar    -122.352813 -122.352806 -122.352800 -122.352794  
Karaoke Bar  -122.320031 -122.320031 -122.320031 -122.320031  
Speakeasy    -122.327395 -122.325465 -122.316779 -122.316737  
Sports Bar   -122.332422 -122.332422 -122.332422 -122.332422  
Wine Bar     -122.339760 -122.334955 -122.327348 -122.313592

### Getting Nightlife data for Seattle

In [108]:
nl_neigh_df2

,categories,commercial,crossStreet,lat,lng,name,neighbourhood,suburb
0,Bar,NaN,at Fremont Ave N,47.649558,-122.349232,Schilling Cider House,NaN,Fremont
1,Brewery,NaN,at Woodland Park Ave N,47.648974,-122.344491,Fremont Brewing Company,NaN,Fremont
3,Brewery,NaN,NaN,47.611565,-122.345212,Cloudburst Brewing,Central Business District,Belltown
5,Wine Bar,NaN,Btwn Stewart And Westlake,47.614940,-122.336999,Le Caviste,Central Business District,Belltown
6,Cocktail Bar,NaN,At E Pine,47.615661,-122.316882,Rachel's Ginger Beer,NaN,Capitol Hill
9,Bar,NaN,at Madison St,47.613396,-122.314286,Diesel,NaN,First Hill
10,Cocktail Bar,NaN,at E Mercer St,47.624605,-122.325434,Sol Liquor Lounge,South Lake Union,Capitol Hill
11,Bar,NaN,Thomas St,47.620559,-122.339892,Gold Bar,South Lake Union,Belltown
12,Bar,NaN,NaN,47.621521,-122.312660,Smith,Montlake,Capitol Hill
13,Bar,NaN,Taylor Ave N,47.618586,-122.346011,TeKu Tavern,South Lake Union,Belltown


In [109]:
nl_neigh_df2.groupby('suburb').nunique()

,categories,commercial,crossStreet,lat,lng,name,neighbourhood,suburb
suburb,,,,,,,,
Belltown,7,1,11,20,20,20,2,1
Capitol Hill,5,0,9,15,15,15,4,1
First Hill,7,0,14,20,20,20,1,1
Fremont,4,0,5,8,8,8,1,1
International District/Chinatown,4,0,5,6,6,6,1,1


In [110]:
# converting the categories to one hot format -- ready for further processing
nl_neigh_onehot=pd.get_dummies(nl_neigh_df2[['categories']], prefix="", prefix_sep="")
nl_neigh_onehot['suburb']=nl_neigh_df2['suburb']
nl_neigh_onehot.head()

,Bar,Brewery,Cocktail Bar,Dive Bar,Gastropub,Hotel Bar,Karaoke Bar,Speakeasy,Sports Bar,Wine Bar,suburb
0,1,0,0,0,0,0,0,0,0,0,Fremont
1,0,1,0,0,0,0,0,0,0,0,Fremont
3,0,1,0,0,0,0,0,0,0,0,Belltown
5,0,0,0,0,0,0,0,0,0,1,Belltown
6,0,0,1,0,0,0,0,0,0,0,Capitol Hill


In [111]:
# grouping the one hot data
nl_neigh_onehot_grp = nl_neigh_onehot.groupby('suburb').mean().reset_index()
nl_neigh_onehot_grp

,suburb,Bar,Brewery,Cocktail Bar,Dive Bar,Gastropub,Hotel Bar,Karaoke Bar,Speakeasy,Sports Bar,Wine Bar
0,Belltown,0.350000,0.100000,0.250000,0.00,0.050000,0.1,0.000000,0.05,0.000000,0.100000
1,Capitol Hill,0.266667,0.000000,0.466667,0.00,0.066667,0.0,0.066667,0.00,0.000000,0.133333
2,First Hill,0.500000,0.050000,0.100000,0.05,0.050000,0.0,0.000000,0.20,0.000000,0.050000
3,Fremont,0.500000,0.250000,0.125000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.125000
4,International District/Chinatown,0.000000,0.166667,0.500000,0.00,0.000000,0.0,0.000000,0.00,0.166667,0.166667


### Get top 5 categories for night life

In [112]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_nl_sorted = pd.DataFrame(columns=columns)
neighborhoods_nl_sorted['suburb'] = nl_neigh_onehot_grp['suburb']

for ind in np.arange(nl_neigh_onehot_grp.shape[0]):
    neighborhoods_nl_sorted.iloc[ind, 1:] = return_most_common_venues(nl_neigh_onehot_grp.iloc[ind, :], num_top_venues)

neighborhoods_nl_sorted.head()

,suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Belltown,Bar,Cocktail Bar,Wine Bar,Hotel Bar,Brewery
1,Capitol Hill,Cocktail Bar,Bar,Wine Bar,Karaoke Bar,Gastropub
2,First Hill,Bar,Speakeasy,Cocktail Bar,Wine Bar,Gastropub
3,Fremont,Bar,Brewery,Wine Bar,Cocktail Bar,Sports Bar
4,International District/Chinatown,Cocktail Bar,Wine Bar,Sports Bar,Brewery,Speakeasy


In [114]:
nl_neigh_onehot_grp.head()

,suburb,Bar,Brewery,Cocktail Bar,Dive Bar,Gastropub,Hotel Bar,Karaoke Bar,Speakeasy,Sports Bar,Wine Bar
0,Belltown,0.350000,0.100000,0.250000,0.00,0.050000,0.1,0.000000,0.05,0.000000,0.100000
1,Capitol Hill,0.266667,0.000000,0.466667,0.00,0.066667,0.0,0.066667,0.00,0.000000,0.133333
2,First Hill,0.500000,0.050000,0.100000,0.05,0.050000,0.0,0.000000,0.20,0.000000,0.050000
3,Fremont,0.500000,0.250000,0.125000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.125000
4,International District/Chinatown,0.000000,0.166667,0.500000,0.00,0.000000,0.0,0.000000,0.00,0.166667,0.166667


In [117]:
# set number of clusters
kclusters = 3

nl_neigh_onehot_grp_clust = nl_neigh_onehot_grp.drop('suburb', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nl_neigh_onehot_grp_clust)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 0])

In [118]:
nl_neigh_onehot_grp_clust.shape

(5, 10)

### Now working on Bellevue Data

In [119]:
# getting 100 nighlife places in Bellevue
search_query='Nightlife'
LIMIT=200
sw='47.56922957853297,-122.27929115295409'
ne='47.66550296313301,-122.06952095031737'

result=getocationData(search_query,LIMIT,sw,ne)
nl_be_df=convert2Df(result)
nl_be_df.head()

,id,name,categories,dislike,ok,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.neighborhood
0,53686cd5498e6d121ef28017,Postdoc Brewing Company,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,False,17625 NE 65th St Ste 100,176th AVE NE,47.663718,-122.105342,"[{'label': 'display', 'lat': 47.66371765098741...",98052,US,Redmond,WA,United States,"[17625 NE 65th St Ste 100 (176th AVE NE), Redm...",90183155,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40b13b00f964a520f6f61ee3,Roanoke Inn,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,False,1825 72nd Ave SE,NaN,47.593981,-122.243354,"[{'label': 'display', 'lat': 47.59398083206654...",98040,US,Mercer Island,WA,United States,"[1825 72nd Ave SE, Mercer Island, WA 98040, Un...",NaN,1905339,https://www.grubhub.com/restaurant/roanoke-inn...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
2,4a83909cf964a52070fb1fe3,THE PUMPHOUSE BAR & GRILL,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",False,False,11802 NE 8th St,at 118th Ave NE,47.617749,-122.183085,"[{'label': 'display', 'lat': 47.61774866370444...",98005,US,Bellevue,WA,United States,"[11802 NE 8th St (at 118th Ave NE), Bellevue, ...",458456015,1643277,https://www.grubhub.com/restaurant/the-pumphou...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,"Wilburton, Bellevue, WA"
3,54275fb7498ea888468fe70b,Tavern Hall,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,False,505 Bellevue Sq,Bellevue Way,47.616588,-122.201711,"[{'label': 'display', 'lat': 47.61658811729160...",98004,US,Bellevue,WA,United States,"[505 Bellevue Sq (Bellevue Way), Bellevue, WA ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4ffb0b30e4b0b8fdabb4a1af,Bellevue Brewing Company,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,False,1820 130th Ave NE Ste 2,at Northup Way,47.626556,-122.166157,"[{'label': 'display', 'lat': 47.62655649637841...",98005,US,Bellevue,WA,United States,"[1820 130th Ave NE Ste 2 (at Northup Way), Bel...",NaN,1587224,https://www.grubhub.com/restaurant/bellevue-br...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN


In [120]:
# cleaning the df
nl_be_filt_df=cleanupDf(nl_be_df)
nl_be_filt_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Postdoc Brewing Company,Brewery,17625 NE 65th St Ste 100,176th AVE NE,47.663718,-122.105342,"[{'label': 'display', 'lat': 47.66371765098741...",98052,US,Redmond,WA,United States,"[17625 NE 65th St Ste 100 (176th AVE NE), Redm...",NaN,53686cd5498e6d121ef28017
1,Roanoke Inn,Bar,1825 72nd Ave SE,NaN,47.593981,-122.243354,"[{'label': 'display', 'lat': 47.59398083206654...",98040,US,Mercer Island,WA,United States,"[1825 72nd Ave SE, Mercer Island, WA 98040, Un...",NaN,40b13b00f964a520f6f61ee3
2,THE PUMPHOUSE BAR & GRILL,Sports Bar,11802 NE 8th St,at 118th Ave NE,47.617749,-122.183085,"[{'label': 'display', 'lat': 47.61774866370444...",98005,US,Bellevue,WA,United States,"[11802 NE 8th St (at 118th Ave NE), Bellevue, ...","Wilburton, Bellevue, WA",4a83909cf964a52070fb1fe3
3,Tavern Hall,Bar,505 Bellevue Sq,Bellevue Way,47.616588,-122.201711,"[{'label': 'display', 'lat': 47.61658811729160...",98004,US,Bellevue,WA,United States,"[505 Bellevue Sq (Bellevue Way), Bellevue, WA ...",NaN,54275fb7498ea888468fe70b
4,Bellevue Brewing Company,Brewery,1820 130th Ave NE Ste 2,at Northup Way,47.626556,-122.166157,"[{'label': 'display', 'lat': 47.62655649637841...",98005,US,Bellevue,WA,United States,"[1820 130th Ave NE Ste 2 (at Northup Way), Bel...",NaN,4ffb0b30e4b0b8fdabb4a1af


In [121]:
nl_be_filt_df.shape

(54, 15)

### Getting Night Club neighbourhoods

In [156]:
def getNeigh2(row):
    lat_lon=str(row['lat'])+", "+str(row['lng'])
    location = geolocator.reverse(lat_lon)
    try:
        row['commercial']=location.raw['address']['commercial']
    except:
        pass
    try:
        row['neighbourhood']=location.raw['address']['neighbourhood']
    except:
        pass
    try:
        row['suburb']=location.raw['address']['suburb']
    except:
#         print(location.raw['address'])
        if 'hamlet' in location.raw['address']:
            row['suburb']=location.raw['address']['hamlet']
    return row

In [157]:
nl_be_neigh_df=nl_be_filt_df.apply(getNeigh2, axis=1)
nl_be_neigh_df.drop(['labeledLatLngs','postalCode','cc','city','state','country','formattedAddress','address','id','neighborhood'], axis=1,\
                    inplace=True)

In [125]:
nl_be_neigh_df.dropna(subset=['suburb']).reset_index(drop=True)
nl_be_neigh_df

,categories,commercial,crossStreet,lat,lng,name,neighbourhood,suburb
0,Brewery,NaN,176th AVE NE,47.663718,-122.105342,Postdoc Brewing Company,NaN,Campton
1,Bar,NaN,NaN,47.593981,-122.243354,Roanoke Inn,NaN,McGilvra
2,Sports Bar,NaN,at 118th Ave NE,47.617749,-122.183085,THE PUMPHOUSE BAR & GRILL,Midlakes,Belridge
3,Bar,NaN,Bellevue Way,47.616588,-122.201711,Tavern Hall,NaN,Ashwood
4,Brewery,NaN,at Northup Way,47.626556,-122.166157,Bellevue Brewing Company,NaN,Belridge
5,Brewery,NaN,at E Lake Sammamish Pkwy NE,47.663594,-122.102816,Mac & Jack's Brewery,NaN,Campton
6,Gastropub,NaN,at NE 8th St,47.618614,-122.201831,Black Bottle Postern,NaN,Ashwood
7,Wine Bar,NaN,in Bellevue Towers,47.614010,-122.198855,Purple Cafe & Wine Bar,Midlakes,Ashwood
8,Brewery,NaN,NaN,47.579836,-122.074262,Big Block Brewing Taproom,NaN,Mint Grove
9,Cocktail Bar,NaN,NaN,47.614311,-122.198669,Lot No. 3,Midlakes,Ashwood


In [126]:
nl_be_neigh_df.groupby('categories').count()

,commercial,crossStreet,lat,lng,name,neighbourhood,suburb
categories,,,,,,,
American Restaurant,0,1,2,2,2,2,2
Arcade,0,1,1,1,1,0,1
Asian Restaurant,0,0,2,2,2,1,2
Bar,0,6,10,10,10,0,10
Brewery,0,3,4,4,4,0,4
Cocktail Bar,0,2,3,3,3,1,3
Dive Bar,1,1,3,3,3,0,2
Gastropub,0,1,1,1,1,0,1
Hotel Bar,0,5,6,6,6,2,5


In [127]:
# only using categories used for Seattle Nightlife
nightlife_places=['Bar', 'Brewery', 'Cocktail Bar', 'Dive Bar', 'Gastropub', 'Hotel Bar', 'Karaoke Bar', 'Speakeasy', 'Sports Bar', 'Wine Bar']
nl_be_neigh_df=nl_be_neigh_df[nl_be_neigh_df['categories'].isin(nightlife_places)]

In [128]:
nl_be_neigh_df

,categories,commercial,crossStreet,lat,lng,name,neighbourhood,suburb
0,Brewery,NaN,176th AVE NE,47.663718,-122.105342,Postdoc Brewing Company,NaN,Campton
1,Bar,NaN,NaN,47.593981,-122.243354,Roanoke Inn,NaN,McGilvra
2,Sports Bar,NaN,at 118th Ave NE,47.617749,-122.183085,THE PUMPHOUSE BAR & GRILL,Midlakes,Belridge
3,Bar,NaN,Bellevue Way,47.616588,-122.201711,Tavern Hall,NaN,Ashwood
4,Brewery,NaN,at Northup Way,47.626556,-122.166157,Bellevue Brewing Company,NaN,Belridge
5,Brewery,NaN,at E Lake Sammamish Pkwy NE,47.663594,-122.102816,Mac & Jack's Brewery,NaN,Campton
6,Gastropub,NaN,at NE 8th St,47.618614,-122.201831,Black Bottle Postern,NaN,Ashwood
7,Wine Bar,NaN,in Bellevue Towers,47.614010,-122.198855,Purple Cafe & Wine Bar,Midlakes,Ashwood
8,Brewery,NaN,NaN,47.579836,-122.074262,Big Block Brewing Taproom,NaN,Mint Grove
9,Cocktail Bar,NaN,NaN,47.614311,-122.198669,Lot No. 3,Midlakes,Ashwood


In [129]:
nl_be_neigh_df.shape

(35, 8)

In [130]:
nl_be_neigh_onehot=pd.get_dummies(nl_be_neigh_df[['categories']], prefix="", prefix_sep="")
nl_be_neigh_onehot['suburb']=nl_be_neigh_df['suburb']
nl_be_neigh_onehot.head()

,Bar,Brewery,Cocktail Bar,Dive Bar,Gastropub,Hotel Bar,Karaoke Bar,Speakeasy,Sports Bar,Wine Bar,suburb
0,0,1,0,0,0,0,0,0,0,0,Campton
1,1,0,0,0,0,0,0,0,0,0,McGilvra
2,0,0,0,0,0,0,0,0,1,0,Belridge
3,1,0,0,0,0,0,0,0,0,0,Ashwood
4,0,1,0,0,0,0,0,0,0,0,Belridge


In [131]:
nl_be_neigh_onehot_grp = nl_be_neigh_onehot.groupby('suburb').mean().reset_index()
nl_be_neigh_onehot_grp

,suburb,Bar,Brewery,Cocktail Bar,Dive Bar,Gastropub,Hotel Bar,Karaoke Bar,Speakeasy,Sports Bar,Wine Bar
0,Ashwood,0.181818,0.00,0.272727,0.00,0.090909,0.181818,0.090909,0.090909,0.00,0.090909
1,Belridge,0.000000,0.25,0.000000,0.25,0.000000,0.000000,0.250000,0.000000,0.25,0.000000
2,Campton,0.000000,1.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Eastgate,0.000000,0.00,0.000000,0.00,0.000000,1.000000,0.000000,0.000000,0.00,0.000000
4,Factoria,0.500000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.50,0.000000
5,Houghton,0.500000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.500000
6,Madison Park,1.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,McGilvra,1.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,Mint Grove,0.000000,1.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Moorland,0.250000,0.00,0.000000,0.00,0.000000,0.500000,0.000000,0.000000,0.00,0.250000


### Get top 5 categories for night life

In [132]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_nl_be_sorted = pd.DataFrame(columns=columns)
neighborhoods_nl_be_sorted['suburb'] = nl_be_neigh_onehot_grp['suburb']

for ind in np.arange(nl_be_neigh_onehot_grp.shape[0]):
    neighborhoods_nl_be_sorted.iloc[ind, 1:] = return_most_common_venues(nl_be_neigh_onehot_grp.iloc[ind, :], num_top_venues)

neighborhoods_nl_be_sorted.head()

,suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ashwood,Cocktail Bar,Hotel Bar,Bar,Wine Bar,Speakeasy
1,Belridge,Sports Bar,Karaoke Bar,Dive Bar,Brewery,Wine Bar
2,Campton,Brewery,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar
3,Eastgate,Hotel Bar,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar
4,Factoria,Sports Bar,Bar,Wine Bar,Speakeasy,Karaoke Bar


In [135]:
nl_be_neigh_onehot_grp.shape

(11, 11)

In [136]:
nl_be_neigh_onehot_grp

,suburb,Bar,Brewery,Cocktail Bar,Dive Bar,Gastropub,Hotel Bar,Karaoke Bar,Speakeasy,Sports Bar,Wine Bar
0,Ashwood,0.181818,0.00,0.272727,0.00,0.090909,0.181818,0.090909,0.090909,0.00,0.090909
1,Belridge,0.000000,0.25,0.000000,0.25,0.000000,0.000000,0.250000,0.000000,0.25,0.000000
2,Campton,0.000000,1.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Eastgate,0.000000,0.00,0.000000,0.00,0.000000,1.000000,0.000000,0.000000,0.00,0.000000
4,Factoria,0.500000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.50,0.000000
5,Houghton,0.500000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.500000
6,Madison Park,1.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,McGilvra,1.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,Mint Grove,0.000000,1.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Moorland,0.250000,0.00,0.000000,0.00,0.000000,0.500000,0.000000,0.000000,0.00,0.250000


### Predicting Clusters for the Bellevue Neighborhoods

In [138]:
nl_be_neigh_onehot_grp_clust = nl_be_neigh_onehot_grp.drop('suburb', 1)

# run k-means clustering
bellevue_clusters=kmeans.predict(nl_be_neigh_onehot_grp_clust)

### Adding cluster column to Bellevue Top 5 table along with neigh lat lon

In [160]:
def getLatLon(row):
    sub=row['suburb']
    location = geolocator.geocode(sub+', WA')
    row['sub_lat']=location.raw['lat']
    row['sub_lon']=location.raw['lon']
    return row

In [165]:
neighborhoods_nl_be_sorted.insert(0, 'Cluster Labels', bellevue_clusters)
nl_be_merged=neighborhoods_nl_be_sorted.apply(getLatLon, axis=1)
nl_be_merged

,Cluster Labels,suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,sub_lat,sub_lon
0,2,Ashwood,Cocktail Bar,Hotel Bar,Bar,Wine Bar,Speakeasy,47.6295435,-122.1954011
1,0,Belridge,Sports Bar,Karaoke Bar,Dive Bar,Brewery,Wine Bar,47.6134327,-122.1676227
2,1,Campton,Brewery,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.6623217,-122.1001226
3,1,Eastgate,Hotel Bar,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.5759181,-122.1398636
4,1,Factoria,Sports Bar,Bar,Wine Bar,Speakeasy,Karaoke Bar,47.5772408,-122.1684296
5,1,Houghton,Wine Bar,Bar,Sports Bar,Speakeasy,Karaoke Bar,40.7837629,-91.6024677
6,1,Madison Park,Bar,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.6359301,-122.2801963
7,1,McGilvra,Bar,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.5909323,-122.2429013
8,1,Mint Grove,Brewery,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.5984335,-122.0817879
9,1,Moorland,Hotel Bar,Wine Bar,Bar,Sports Bar,Speakeasy,42.441639,-94.2949662


### Adding cluster column to Seattle Top 5 table along with neigh lat lon

In [167]:
neighborhoods_nl_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
nl_merged=neighborhoods_nl_sorted.apply(getLatLon, axis=1)
nl_merged

,Cluster Labels,suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,sub_lat,sub_lon
0,1,Belltown,Bar,Cocktail Bar,Wine Bar,Hotel Bar,Brewery,47.6132311,-122.3453607
1,2,Capitol Hill,Cocktail Bar,Bar,Wine Bar,Karaoke Bar,Gastropub,47.6238307,-122.3183689
2,1,First Hill,Bar,Speakeasy,Cocktail Bar,Wine Bar,Gastropub,47.6093053,-122.3249839
3,1,Fremont,Bar,Brewery,Wine Bar,Cocktail Bar,Sports Bar,40.7345943,-95.5924782
4,0,International District/Chinatown,Cocktail Bar,Wine Bar,Sports Bar,Brewery,Speakeasy,47.598496299999994,-122.32799987174627


In [169]:
# merging the 2 df for visualization
nl_data=pd.concat([nl_be_merged,nl_merged],ignore_index=True)
nl_data

,Cluster Labels,suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,sub_lat,sub_lon
0,2,Ashwood,Cocktail Bar,Hotel Bar,Bar,Wine Bar,Speakeasy,47.6295435,-122.1954011
1,0,Belridge,Sports Bar,Karaoke Bar,Dive Bar,Brewery,Wine Bar,47.6134327,-122.1676227
2,1,Campton,Brewery,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.6623217,-122.1001226
3,1,Eastgate,Hotel Bar,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.5759181,-122.1398636
4,1,Factoria,Sports Bar,Bar,Wine Bar,Speakeasy,Karaoke Bar,47.5772408,-122.1684296
5,1,Houghton,Wine Bar,Bar,Sports Bar,Speakeasy,Karaoke Bar,40.7837629,-91.6024677
6,1,Madison Park,Bar,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.6359301,-122.2801963
7,1,McGilvra,Bar,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.5909323,-122.2429013
8,1,Mint Grove,Brewery,Wine Bar,Sports Bar,Speakeasy,Karaoke Bar,47.5984335,-122.0817879
9,1,Moorland,Hotel Bar,Wine Bar,Bar,Sports Bar,Speakeasy,42.441639,-94.2949662


### Visualizing the resulting Clusters

In [179]:
lon

'-122.1954011'

In [199]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nl_data['sub_lat'], nl_data['sub_lon'], nl_data['suburb'], nl_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1]#,
#         fill_opacity=0.7
    ).add_to(map_clusters)
       
map_clusters

In [181]:
nl_merged

,Cluster Labels,suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,sub_lat,sub_lon
0,1,Belltown,Bar,Cocktail Bar,Wine Bar,Hotel Bar,Brewery,47.6132311,-122.3453607
1,2,Capitol Hill,Cocktail Bar,Bar,Wine Bar,Karaoke Bar,Gastropub,47.6238307,-122.3183689
2,1,First Hill,Bar,Speakeasy,Cocktail Bar,Wine Bar,Gastropub,47.6093053,-122.3249839
3,1,Fremont,Bar,Brewery,Wine Bar,Cocktail Bar,Sports Bar,40.7345943,-95.5924782
4,0,International District/Chinatown,Cocktail Bar,Wine Bar,Sports Bar,Brewery,Speakeasy,47.598496299999994,-122.32799987174627


In [197]:
# merging clustered neighborhoods with night club counts
nl_merged2=pd.merge(nl_merged,nc_neigh_df.groupby('suburb').agg({'name':'count'}),on='suburb')
nl_merged2[['Cluster Labels','suburb','name']].sort_values(by=['Cluster Labels'])
# name = count of Nigh Clubs

,Cluster Labels,suburb,name
4,0,International District/Chinatown,13
0,1,Belltown,5
2,1,First Hill,15
3,1,Fremont,2
1,2,Capitol Hill,3


The above result says Cluster 0 is prolly a very good Neighborhood for Night Clubs <br>
Cluster 1 Neighborhoods seems like the next best obtion <br>
Cluster 2 Neighborhoods is prolly the last the place to start such business <br>

In [198]:
nl_be_merged[['Cluster Labels','suburb']].sort_values(by=['Cluster Labels'])

,Cluster Labels,suburb
1,0,Belridge
2,1,Campton
3,1,Eastgate
4,1,Factoria
5,1,Houghton
6,1,Madison Park
7,1,McGilvra
8,1,Mint Grove
9,1,Moorland
10,1,Overlake


**Cluster 0**:  **Belridge** has the highest correlation with Chinaton and seems like the bes place to setup Night Club <br>
**Cluster 1**: We have a bunch of secondary neighborhoods around bellevue where we can plan night clubs <br>
**Cluster 2**: We only have 1 Neighborhood in cluster 2 with few Nighclubs hence the least priority on the list <br>

**Note:** 
1. The above findings are solely based on the Four squary data for Night life
2. Since Neighborhood data was sparse we used alternative fields like suburbs and hamlets which provided more consistent data and seem to be consistent with Neighborhoods